In [34]:
import os
import cv2
import time
import json
import random
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
import matplotlib.pyplot as plt


from PIL import Image
from enum import Enum
from glob import glob
from tqdm import trange
from pprint import pprint
from colorama import Fore
from tqdm import tqdm_notebook
from sklearn.utils import shuffle
from IPython.display import display


from sklearn.metrics import *
from sklearn.preprocessing import *
from sklearn.model_selection import *


# Neural Network Models
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.layers import Input, Concatenate, BatchNormalization
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, Dropout


# Regression Models
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor,VotingRegressor, AdaBoostRegressor, GradientBoostingRegressor, BaggingRegressor

warnings.filterwarnings("ignore")
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [35]:
# Set random seed to enable re-production

def set_random():
    np.random.seed(100)
    random.seed(100)
    tf.random.set_seed(100)
    os.environ['PYTHONHASHSEED'] = '100'

In [36]:
# Generate data folds for cross validation

def fold_generator(data, target='Pawpularity', cv = 5):
    
    # Fold generator
    kf = StratifiedKFold(n_splits=cv)
    
    # Shuffle the dataset to generate folds
    # Since the Pawpularity ranges from 1 to 100, we classify the scores into 10 levels
    data = shuffle(data).reset_index(drop=True)
    n_bins = 10
    data['bins'] = data[target].map(lambda x: int(x/10))
    
    # Stratified sampling to construct folds
    for fid, (train_idx, valid_idx) in enumerate(kf.split(X=data, y=data['bins'])):
        data.loc[valid_idx, 'Fold'] = fid
        
    data = data.drop(["bins"], axis = 1)         
    return data 

In [37]:
# Self-defined cross validation function

def cross_valid(df, model, cv = 5):
    
    X = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
    Y = 'Pawpularity'

    r2sr_train = 0
    r2sr_valid = 0
    rmse_train = 0
    rmse_valid = 0
    
    # cv: the number of folds
    for fold in range(cv):
        train_df = df.loc[df['Fold'] != fold].reset_index(drop = True)
        valid_df = df.loc[df['Fold'] == fold].reset_index(drop = True)

        train_X = train_df[X]
        train_Y = train_df[Y]

        valid_X = valid_df[X]
        valid_Y = valid_df[Y]
        
        if type(model).__name__ == 'CatBoostRegressor':
            model.fit(train_X, train_Y, verbose=False)
        else:
            model.fit(train_X, train_Y)

        train_pred = model.predict(train_X)
        valid_pred = model.predict(valid_X)

        rmse_train += np.sqrt(mean_squared_error(train_Y, train_pred))
        rmse_valid += np.sqrt(mean_squared_error(valid_Y, valid_pred))
        
    return rmse_train/cv, rmse_valid/cv

In [38]:
# Reshape image

def image_reshape(image_path, image_size):
    # Load image
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    
    # Resize to regularize the input shape
    img = tf.image.resize(img, (image_size, image_size))
    
    # Normalize channels to range (0, 1)
    img = tf.cast(img, tf.float32) / 255.0
    
    return img

In [39]:
# Reshape image

def load_image(is_trainset):
    def just_reshape(img_path):
        img = tf.io.read_file(img_path)
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.cast(img, tf.float32)
        img = tf.image.resize(img, (224, 224))
        img = tf.keras.applications.efficientnet.preprocess_input(img) 
        return img
    
    def reshape_with_label(img_path, label):
        return just_reshape(img_path), label

    return reshape_with_label if is_trainset else just_reshape

In [40]:
# Image augmentation

def augment_image(is_trainset):
    def just_augment(img):
        img = tf.image.random_flip_left_right(img)
        # img = tf.image.random_flip_up_down(img)
        img = tf.image.random_contrast(img, 0.95, 1.05)
        img = tf.image.random_saturation(img, 0.95, 1.05)
        return img
    
    def augment_with_label(img, label):
        return just_augment(img), label
    
    return augment_with_label if is_trainset else just_augment

In [41]:
# Use TensorFlow Database to speed up dataset processing

def construct_tf_dataset(df, batch_size, is_trainset = False, 
                         use_augmentation = False, use_shuffle = False):
    load_image_function = load_image(is_trainset)
    augment_image_function = augment_image(is_trainset)
    
    if is_trainset:
        dataset = tf.data.Dataset.from_tensor_slices((df['Path'].values, df['Pawpularity'].values))
        dataset = dataset.map(load_image_function, num_parallel_calls=AUTOTUNE)
    else:
        dataset = tf.data.Dataset.from_tensor_slices((df['Path'].values))
        dataset = dataset.map(load_image_function, num_parallel_calls=AUTOTUNE)
        
    if use_augmentation:
        dataset = dataset.map(augment_image_function, num_parallel_calls=AUTOTUNE)
    
    if use_shuffle:
        dataset = dataset.shuffle(1000, reshuffle_each_iteration=True)
        
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(AUTOTUNE)
    return dataset

In [42]:
# Set random seed to enable re-production

set_random()

In [43]:
# Dataset path

train_csv = "../input/petfinder-pawpularity-score/train.csv"
test_csv = "../input/petfinder-pawpularity-score/test.csv"
submission_csv = "../input/petfinder-pawpularity-score/submission.csv"

train_dir = "../input/petfinder-pawpularity-score/train"
test_dir = "../input/petfinder-pawpularity-score/test"

In [44]:
# Load csv and image path to dataframe

data_df = pd.read_csv(train_csv)
test_df = pd.read_csv(test_csv)

data_df['Path'] = data_df['Id'].apply(lambda x : train_dir + '/' + x + '.jpg')
test_df['Path'] = test_df['Id'].apply(lambda x : test_dir + '/' + x + '.jpg')

In [45]:
# Test

display(data_df)

In [46]:
# Cross validation fold generation

data_df_cv = fold_generator(data_df, target = 'Pawpularity', cv = 5)

In [47]:
# Test

display(data_df_cv)

In [48]:
# Result set

Result_set = {
    "Model" : [],
    "rmse_train" : [],
    "rmse_valid" : []
}

In [53]:
# # Reshape image
image_size = 128
x_data = []
x_test = []

for i, img in tqdm_notebook(enumerate(data_df['Path']), total=len(data_df)):
    x_data.append(image_reshape(img, image_size))
    
for i, img in tqdm_notebook(enumerate(test_df['Path']), total=len(test_df)):
    x_test.append(image_reshape(img, image_size))

In [54]:
# # Generate train and valid set

x_data = np.array(x_data)
x_test = np.array(x_test)
y_data = data_df['Pawpularity']

trn_X, val_X, trn_Y, val_Y = train_test_split(x_data, y_data, test_size=0.2)

In [55]:
# # Build CNN model
K.clear_session()

nn = Sequential()
nn.add(Conv2D(8 , (3,3), (2,2), activation='relu', padding='same', input_shape=(128,128,3)))
nn.add(Conv2D(16, (3,3), (2,2), activation='relu', padding='same'))
nn.add(Conv2D(32, (3,3), (2,2), activation='relu', padding='same'))
nn.add(Flatten())
nn.add(Dense(units=128, activation='relu'))
nn.add(Dropout(rate=0.5, seed=2021))
nn.add(Dense(units=1))

nn.summary()

In [67]:
# # Early stopping callback function

early_stop = tf.keras.callbacks.EarlyStopping(patience=15)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2, min_lr=1e-9)
callbacks = [early_stop, reduce_lr]

In [68]:
# # Train CNN model32

nn.compile(loss='mse', optimizer='Adam', 
        metrics=[tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

history = nn.fit(trn_X, trn_Y, epochs=25, batch_size=128, 
                validation_data = (val_X, val_Y), callbacks=callbacks)

In [69]:

cbr_cnn = CatBoostRegressor(iterations=200, learning_rate=0.01)
_, _ = cross_valid(data_df_cv, cbr_cnn, cv = 5)

In [76]:
# # Prediction, use model nn

X = ['Subject Focus', 'Eyes', 'Face', 'Near', 'Action', 'Accessory', 'Group', 'Collage', 'Human', 'Occlusion', 'Info', 'Blur']
test_X = test_df[X]

pred_cnn = nn.predict(x_test, verbose = False)
pred_cbr_cnn = cbr_cnn.predict(test_X).reshape(8, 1)

submission_cnn = pd.DataFrame()
submission_cnn['Id'] = test_df['Id']
submission_cnn['Pawpularity'] = (pred_cnn + pred_cbr_cnn)/2.0

submission_cnn.to_csv("method-2-submission.csv", index = False)

print(submission_cnn)